In [1]:
import pymongo as pm
from pymongo import MongoClient
from bson import Code
# client = MongoClient('localhost', 27017)
client = MongoClient('mongodb://localhost:27017/')
stockDB = client.stock
companyDB = client.companyData
stockCol = stockDB.stocks
companyCol = companyDB.companies

import warnings
warnings.filterwarnings('ignore')

We'll be using pymongo to query mongodb, however the js commands has also been included for each operation as docstring. 
If no docstring, please do yourself

In [2]:
#lets see the structure of each collection

"""
db.stocks.findOne()
"""
stockCol.find_one()


{'_id': ObjectId('4d094f58c96767d7a0099d49'),
 'exchange': 'NASDAQ',
 'stock_symbol': 'AACC',
 'date': '2008-03-07',
 'open': 8.4,
 'high': 8.75,
 'low': 8.08,
 'close': 8.55,
 'volume': 275800,
 'adj close': 8.55}

In [3]:
"""
db.companies.findOne()
"""
companyCol.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'permalink': 'abc3',
 'crunchbase_url': 'http://www.crunchbase.com/company/adventnet',
 'homepage_url': 'http://adventnet.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'manageengine',
 'category_code': 'enterprise',
 'number_of_employees': 600,
 'founded_year': 1996,
 'deadpooled_year': 2,
 'tag_list': '',
 'alias_list': 'Zoho ManageEngine ',
 'email_address': 'pr@adventnet.com',
 'phone_number': '925-924-9500',
 'description': 'Server Management Software',
 'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22),
 'updated_at': 'Wed Oct 31 18:26:09 UTC 2012',
 'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
 'image': {'avail

In [33]:
%%time
#get distinct keys in each collection
#we can use map reduce in mongodb. the 'map' function operates on all doc that matches optional 'query' condition we can pass.
#reduce will receive a key and a single value or list of values on which we can do aggregation. 
#more at  https://docs.mongodb.com/manual/core/map-reduce/
#Note : "myresults" is actually a collection which stores the results of reduce which will be used for other functions like finalize

map_ = Code("function() { for (var key in this) { emit(key, null); } }")
reduce_ = Code("function(key, stuff) { return null; }")
result = stockCol.map_reduce(map_, reduce_, "myresults")
print(result)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'stock'), 'myresults')
Wall time: 3min 30s


In [35]:
%%time

#aggregation pipeline works similar to mapreduce but it gives a nice way to construct our pipeline in a easy understandable way.

"""
db.stocks.aggregate([{ "$project": {
    "data": { "$objectToArray": "$$ROOT" }
  }},
  { "$project": { "data": "$data.k" }},
  { "$unwind": "$data" },
  { "$group": {
    "_id": null,
    "keys": { "$addToSet": "$data" }}}]).pretty()

"""

distinctKeyPipeline = [
  { "$project": {
    "data": { "$objectToArray": "$$ROOT" }
  }},
  { "$project": { "data": "$data.k" }},
  { "$unwind": "$data" },
  { "$group": {
    "_id": None,
    "keys": { "$addToSet": "$data" }}}]

distinctKeys = [doc for doc in stockCol.aggregate(distinctKeyPipeline)]
distinctKeys

Wall time: 42.4 s


[{'_id': None,
  'keys': ['high',
   'close',
   'open',
   '_id',
   'low',
   'date',
   'adj close',
   'stock_symbol',
   'volume',
   'exchange']}]

In [2]:
#Lets see how many docs has the field 'products' in companies collection
companyCol.find({"products" : {"$exists": True}}).count()

18801

In [3]:
#all the docs has the field but only 5519 has some value
"""
we can use [], '', None (pymongo) or null in (mongo cli) to check how many docs has actual values
"""
companyCol.find({"products" : {"$exists": True, "$ne":[]}}).count()

5519

In [45]:
#how many docs in each collection
"""
db.stocks.count()
"""
print("Stock collection: ", stockCol.count())
print("Companies collection: ", companyCol.count())

Stock collection:  4308303
Companies collection:  18801


In [51]:
#how many distinct stock_symbol we have
"""
db.stocks.distinct('stock_symbol').length
"""
len(stockCol.distinct('stock_symbol'))

1618

In [54]:
#how many distinct titles in companies collection. we can access the nested elements easily in mongodb
"""
db.companies.distinct('relationships.title').length
"""
len(companyCol.distinct('relationships.title'))

30205

In [6]:
#we have more than 30K values. just quickly test that we can access distinct elements from list.
"""
db.companies.distinct('relationships.is_past').length
"""
companyCol.distinct('relationships.is_past')

[None, False, True]

In [56]:
#nested doc can be accessed even further
"""
db.companies.distinct('relationships.person.first_name').length
"""
len(companyCol.distinct('relationships.person.first_name'))

9218

In [64]:
#find how many companies has "CEO and Co-Founder" role
"""
db.companies.find({"relationships.title": "CEO and Co-Founder"}).count()
"""

len(list(companyCol.find({"relationships.title": "CEO and Co-Founder"})))

55

In [66]:
#above command produces too much of strings, lets print or "project" just the company names that has the role.
#in the below if we set name : 0 , then all keys exept name. and we can't mix 0 and 1's here. _id will be default and explicitly mention 0
"""
db.companies.find({"relationships.title": "CEO and Co-Founder"}, {"name": 1})
"""

list(companyCol.find({"relationships.title": "CEO and Co-Founder"}, {"name" : 1, "_id":0}))


[{'name': 'AdventNet'},
 {'name': 'Prosper'},
 {'name': 'Mashery'},
 {'name': 'Meebo'},
 {'name': 'introNetworks'},
 {'name': 'ZYB'},
 {'name': 'Pluck'},
 {'name': 'Lastminute'},
 {'name': 'Dopplr'},
 {'name': 'Mobovivo'},
 {'name': 'JackBe'},
 {'name': 'BitGravity'},
 {'name': 'Eurekster'},
 {'name': 'Pelago'},
 {'name': 'Omniture'},
 {'name': 'jellyfish'},
 {'name': 'myfirstpaycheck'},
 {'name': 'XO Group'},
 {'name': 'Skydeck'},
 {'name': 'Omnisio'},
 {'name': 'AnchorFree'},
 {'name': 'iovation'},
 {'name': 'Friend2Friend'},
 {'name': 'deviantART'},
 {'name': 'IndieFlix'},
 {'name': 'Musicbank'},
 {'name': 'Limbo'},
 {'name': 'Forticom'},
 {'name': 'Vitrium Systems'},
 {'name': 'Shapeways'},
 {'name': 'LivingSocial'},
 {'name': 'iloho'},
 {'name': 'BizFilings'},
 {'name': 'Livebookings'},
 {'name': 'Interspire'},
 {'name': 'Fitbit'},
 {'name': 'VideoSurf'},
 {'name': 'Gamelet'},
 {'name': 'Interspire'},
 {'name': 'Fitbit'},
 {'name': 'VideoSurf'},
 {'name': 'Gamelet'},
 {'name': 'Na

In [68]:
list(companyCol.find({"name": "PBworks"}))[0]

{'_id': ObjectId('52cdef7c4bab8bd675297e03'),
 'name': 'PBworks',
 'permalink': 'pbworks',
 'crunchbase_url': 'http://www.crunchbase.com/company/pbworks',
 'homepage_url': 'http://www.pbworks.com',
 'blog_url': 'http://blog.pbworks.com/',
 'blog_feed_url': 'http://blog.pbworks.com/feed/',
 'twitter_username': 'PBworks',
 'category_code': 'enterprise',
 'number_of_employees': 25,
 'founded_year': 2005,
 'founded_month': 6,
 'founded_day': 1,
 'deadpooled_year': None,
 'deadpooled_month': None,
 'deadpooled_day': None,
 'deadpooled_url': None,
 'tag_list': 'wiki, social, collaboration, productivity, enterprise, cloud',
 'alias_list': '',
 'email_address': 'sales@pbworks.com',
 'phone_number': '415-963-4369',
 'description': 'Hosted Collaboration Provider',
 'created_at': 'Sat Jul 14 09:55:02 UTC 2007',
 'updated_at': 'Fri Oct 25 12:12:18 UTC 2013',
 'overview': '<p>With over a million communities using PBworks to collaborate publicly and privately, PBworks is the largest business and edu

In [70]:
#finding companies that has either of the roles "Director of Engineering", "CEO and Co-Founder"

len(list(companyCol.find({"relationships.title": {"$in" : ["Director of Engineering", "CEO and Co-Founder"]}}, {"name": 1})))

196

In [17]:
#finding companies that has the role "Director of Engineering" and founding year is > 2005

len(list(companyCol.find({"$and" : [ {"relationships.title" : "Director of Engineering" }, 
                                    {"founded_year" : {"$gt" : 2005}}]}, {"name": 1})))

46

In [18]:
#same as above, without using "$and"

len(list(companyCol.find({"relationships.title": "Director of Engineering","founded_year" : {"$gt" : 2005}}, {"name": 1})))

46

In [22]:
#however if we're using "and" on the same field, then we have to use "$and", else the last condition will replace all other conditions

len(list(companyCol.find({"$and" : [ {"relationships.title" : "Director of Engineering" }, 
                                    {"relationships.title" :"CEO and Co-Founder"}]}, {"name": 1})))

2

In [23]:
len(list(companyCol.find({"relationships.title": "Director of Engineering","relationships.title": "CEO and Co-Founder"}, {"name": 1})))

55

In [24]:
#following is the same as above

len(list(companyCol.find({"relationships.title": "CEO and Co-Founder"}, {"name": 1})))

55

In [25]:
#we can use "$all" as well instead of "$and"

len(list(companyCol.find({"relationships.title" : {"$all" : ["Director of Engineering","CEO and Co-Founder"]}}, {"name": 1})))

2

In [29]:
#lets find the companies that has "Sales Director" role and is_past = False

list(companyCol.find({"relationships.title": "Sales Director","relationships.is_past" : False}, {"name": 1,
                                                                                                              "relationships.title" : 1,
                                                                                                             "relationships.is_past":1}))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'relationships': [{'is_past': False,
    'title': 'Founder and CEO, Board Of Directors'},
   {'is_past': False, 'title': 'CFO'},
   {'is_past': False, 'title': 'COO'},
   {'is_past': False, 'title': 'Vice- President and General Counsel'},
   {'is_past': False, 'title': 'VP of Engineering & Product'},
   {'is_past': False, 'title': 'VP, Global Advertising Sales'},
   {'is_past': False,
    'title': 'Vice President Advertising and Global Operations'},
   {'is_past': False, 'title': 'VP, EMEA'},
   {'is_past': False, 'title': 'VP, Advertising'},
   {'is_past': False, 'title': 'VP Human Resources, Recruiting'},
   {'is_past': False, 'title': 'VP of Engineering'},
   {'is_past': False, 'title': 'VP Product'},
   {'is_past': False, 'title': 'VP Engineering & Products'},
   {'is_past': False, 'title': 'VP of Partnerships and Platform Marketing'},
   {'is_past': False, 'title': 'Vice President, Engineering'},
   {'is_past':

As we can see this is not quite the result we want, and above query is equal to an AND condition. What we want is each element in the array should meet both the condition

In [33]:
#if we want to match filter all conditions on a single array element, then we can use "$elemMatch". So there are 19 companies with "Sales Director" right now.

len(list(companyCol.find({"relationships" : {"$elemMatch": {"title": "Sales Director","is_past" : False}}}, {"name": 1,
                                                                                                              "relationships.title" : 1,
                                                                                                             "relationships.is_past":1})))

19

In [40]:
#querying companies with only 5 elements in relationships array

len(list(companyCol.find({"relationships" :  { "$size" : 5}}, {"name": 1,"relationships.title" : 1})))

720

Couple more complex queries

In [80]:
list(companyCol.find({"founded_year" : {"$gt" : 2010} , "number_of_employees" : {"$gte" : 50}}, {"founded_year": 1,
                                                                                                  "number_of_employees" : 1,
                                                                                                 "name" : 1}))

[{'_id': ObjectId('52cdef7c4bab8bd67529801f'),
  'name': 'Wamba',
  'number_of_employees': 120,
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd6752982d4'),
  'name': 'Advaliant',
  'number_of_employees': 60,
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529831a'),
  'name': 'Social Gaming Network',
  'number_of_employees': 100,
  'founded_year': 2011},
 {'_id': ObjectId('52cdef7c4bab8bd6752983b6'),
  'name': 'Mobiluck',
  'number_of_employees': 1234,
  'founded_year': 2012},
 {'_id': ObjectId('52cdef7d4bab8bd675299dae'),
  'name': 'Navara',
  'number_of_employees': 80,
  'founded_year': 2012},
 {'_id': ObjectId('52cdef7e4bab8bd67529b2f3'),
  'name': '4shared',
  'number_of_employees': 666,
  'founded_year': 2011},
 {'_id': ObjectId('52cdef7f4bab8bd67529c6d4'),
  'name': 'Treehouse',
  'number_of_employees': 55,
  'founded_year': 2011}]

In [84]:
list(companyCol.find({"$and" : [ {"relationships.title" : "Director of Engineering" }, 
                                {"relationships.title" :"CEO and Co-Founder"}],"founded_year" : {"$gt" : 1990} ,
                      "number_of_employees" : {"$gte" : 50}}, {"founded_year": 1,"number_of_employees" : 1,"name" : 1}))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'number_of_employees': 600,
  'founded_year': 1996}]

In [41]:
#lets query in stocks collection,

stockCol.find_one()

{'_id': ObjectId('4d094f58c96767d7a0099d49'),
 'exchange': 'NASDAQ',
 'stock_symbol': 'AACC',
 'date': '2008-03-07',
 'open': 8.4,
 'high': 8.75,
 'low': 8.08,
 'close': 8.55,
 'volume': 275800,
 'adj close': 8.55}

In [51]:
#lets find stocks which close value is more than the open value.
# $expr is useful command when we want to compare two fields within the same doc

print("Close > Open  :  ",len(list(stockCol.find( { "$expr": { "$gt": [ "$close" , "$open" ] } , "stock_symbol" : "AACC"} ))))
print("Open > Close  :  ",len(list(stockCol.find( { "$expr": { "$gt": [ "$open" , "$close" ] } , "stock_symbol" : "AACC"} ))))
print("Open = Close  :  ",len(list(stockCol.find( { "$expr": { "$eq": [ "$open" , "$close" ] } , "stock_symbol" : "AACC"} ))))

Close > Open  :   499
Open > Close  :   495
Open = Close  :   22


In [60]:
#we can use the where clause to get the same logic, in addition to that, we can perform complex queries like checking the sum of two fields satisfying a condition.

len(list(stockCol.find({"stock_symbol" : "AACC", "$where" : """function() {return this.close > this.open}""" })))

# we can check like this as well
# len(list(stockCol.find({"stock_symbol" : "AACC", "$where" : """function() {return this.close + this.open > 25}""" })))

866

In [58]:
%%time
len(list(stockCol.find( { "$expr": { "$gt": [ "$close" , "$open" ] } } )))

Wall time: 32.4 s


1656436

In [59]:
%%time
len(list(stockCol.find({"$where" : """function() {return this.close > this.open}""" })))

Wall time: 2min 46s


1656436

As we can see, where takes a lot of time compared to expr, this is because the function we submitted is executed on all the documents.
More here, https://docs.mongodb.com/manual/reference/operator/query/where/

Let's do some text matches 

In [63]:
companyCol.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'permalink': 'abc3',
 'crunchbase_url': 'http://www.crunchbase.com/company/adventnet',
 'homepage_url': 'http://adventnet.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'manageengine',
 'category_code': 'enterprise',
 'number_of_employees': 600,
 'founded_year': 1996,
 'deadpooled_year': 2,
 'tag_list': '',
 'alias_list': 'Zoho ManageEngine ',
 'email_address': 'pr@adventnet.com',
 'phone_number': '925-924-9500',
 'description': 'Server Management Software',
 'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22),
 'updated_at': 'Wed Oct 31 18:26:09 UTC 2012',
 'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
 'image': {'avail

In [75]:
#so there are 1241 companies that starts with A

len(list(companyCol.find({"name": {"$regex" : "^A"}}, {"name" : 1})))

1241

In [72]:
len(list(companyCol.find({"name": {"$regex" : "^a"}}, {"name" : 1})))

71

As we can see regex is case sensitive. we can optionally use the param $options to make the search case insensitive

In [77]:
len(list(companyCol.find({"name": {"$regex" : "^a", '$options' : 'i'}}, {"name" : 1})))

1312

In [82]:
#we can check for the word software in the overview field of documents 

len(list(companyCol.find({"overview": {"$regex" : "software", '$options' : 'i'}}, {"name" : 1, "overview":1})))

2927

In [84]:
list(companyCol.find({"$and" : [{"overview": {"$regex" : "software", '$options' : 'i'}},
                                   {"overview": {"$regex" : "marketing", '$options' : 'i'}}]}, {"name" : 1, "overview":1}))

[{'_id': ObjectId('52cdef7c4bab8bd675297dda'),
  'name': 'iContact',
  'overview': '<p>iContact is the largest private provider of email marketing and social media marketing software and services to small and mid-sized companies globally. It is based in Morrisville, NC. iContact makes it easy to create, send, and track email newsletters, surveys, autoresponders. iContact&#8217;s products include iContact, iContact Enterprise, iContact Agency, iContact for Salesforce, iContact Free Edition and Ettend.com. </p>\n\n<p>As a B Corporation, iContact utilizes the 4-1s Corporate Social Responsibility Model, donating 1% of employee time to community volunteering, 1% of payroll, 1% of equity, and 1% of product to its local and global community as part of its social mission. iContact works hard to maintain a fun, creative, energetic, challenging, and community-oriented company culture. </p>\n\n<p>iContact was founded in 2003 by Ryan Allis and Aaron Houghton in Chapel Hill, NC.</p>'},
 {'_id': Obj